# 保存した教材バックアップをリストアします
<HR>

保存してある教材のバックアップファイルを展開して、指定された位置に配置します。

# リストアの準備

このNotebookを実行すると、ホームディレクトリ直下の下記のフォルダを、バックアップファイルの内容で上書きコピーします。

**上書きしたくないファイル/フォルダは、下記のフォルダから退避しておいて下さい。**

* `textbook/`
  + 教材Notebookおよび教材Notebookから呼び出すスクリプトなど
* `info/`
  + 受講者が参照する参考資料など
* `draft/`
  + 教材以外で保存して置きたいNotebook、その他ファイル
  
バックアップを作成する際に、上記以外のファイル/フォルダを `INCLUDES` で追加した場合、それらのファイル/フォルダも上書きコピーされます。

# バックアップファイルを転送します

リストアする バックアップファイル（`*.tgz` または `*.tar.gz` 形式）を、Jupyter に転送して下さい。

バックアップファイルがローカルマシンにある場合は、下記の手順でアップロードできます。
  1. ブラウザのHome画面で、フォルダを開きます。
  2. 画面右上の［Upload］ボタンを押します。ファイル選択ダイアログが開きます。
  3. ファイルを選択して［開く］ボタンを押します。ファイル一覧最上部にアップロードファイル名が表示されます。
  4. ファイル名を変更する場合にはファイル名を選択して編集します。
  5. バックアップファイル名の行の右端の［Upload］ボタンを押すとアップロードされます。

# バックアップファイル名を指定します

* ARCHIVE_FILE_NAME にリストアするバックアップファイル名を指定して下さい。
```
%env ARCHIVE_FILE_NAME=backup-course-01.tgz
```
* バックアップファイルは、ホームディレクトリからの相対パスで指定します。

**次のセルでバックアップファイル名を指定して、セルを実行して下さい。**

In [ ]:
%env ARCHIVE_FILE_NAME=backup-course-01.tgz

# 次のセルを実行して、バックアップファイルを展開して下さい


バックアップファイルを展開して、ホームディレクトリに配置します。

In [ ]:
%%bash
set -e

umask 0022
cd

if [ ! -e "${ARCHIVE_FILE_NAME}" ]; then
  echo "${ARCHIVE_FILE_NAME}: No such file or directory" 1>&2
  exit 1
fi

# Test Archive
tar -tz -f "${ARCHIVE_FILE_NAME}" >/dev/null
if [ $? -ne 0 ]; then
  echo "${ARCHIVE_FILE_NAME}: Unexpected backup file format" 1>&2
  exit 1
fi 

# Extract files
tar -xz --overwrite -f "${ARCHIVE_FILE_NAME}"
if [ $? -ne 0 ]; then
  echo "Error while extracting backup file"
  exit 1
fi

# Trust notebooks
tar -tz -f "${ARCHIVE_FILE_NAME}" | while read -r path; do
  if [ -f "${path}" -a "${path: -6}" = ".ipynb" ]; then
    jupyter-trust "${path}"
  fi
done

echo
echo "Successfully restored."